In [29]:
import os
import sys
import json
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [227]:
from keras import backend as K
from sklearn.metrics import precision_recall_fscore_support

In [41]:
with open("D:\\PESU\\Sem_V\\AI\\_Lab_\\Lab1\\pos_amazon_cell_phone_reviews.json") as f:
    data = json.load(f)
data = data['root']
#print(len(data))
df_pos = pd.DataFrame(data)
summaries_positive = []
text_positive = []
for i in data:
    summaries_positive.append(i['summary'])
    text_positive.append(i['text'])
label = [1] * 108664
df_pos['label'] = label

108664


In [47]:
df_pos2 = df_pos[0:1625]

In [49]:
with open("D:\\PESU\\Sem_V\\AI\\_Lab_\\Lab1\\neg_amazon_cell_phone_reviews.json") as f:
    data = json.load(f)
summaries_negative=[]
text_negative=[]
data = data['root']
df_neg = pd.DataFrame(data)
for i in data:
    summaries_negative.append(i['summary'])
    text_negative.append(i['text'])
label = [0] * len(data)
df_neg['label'] = label

In [51]:
df_neg2 = df_neg[0:1625]

In [58]:
df = df_pos2.append(df_neg2)

In [62]:
seed = 123

In [63]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.076, random_state=seed)

In [130]:
tokenizer = Tokenizer()

In [131]:
summary = list(X_train['summary'])
summary_test = list(X_test['summary'])

In [132]:
tokenizer.fit_on_texts(summary)
vocab = tokenizer.word_index
vocab_counts = tokenizer.word_counts

In [133]:
sequences = tokenizer.texts_to_sequences(summary)
sequences_test = tokenizer.texts_to_sequences(summary_test)

In [134]:
len(max(sequences_test, key = lambda x: len(x)))

20

58

In [136]:
#pad_length = len(max(sequences, key = lambda x: len(x)))
pad_length = 22

In [137]:
pad_length

22

In [138]:
new_X_train = pad_sequences(sequences, maxlen = pad_length)
new_X_test =  pad_sequences(sequences_test, maxlen = pad_length)

In [139]:
new_X_train

array([[   0,    0,    0, ...,  110,  360,   36],
       [   0,    0,    0, ...,   90,   76,  106],
       [   0,    0,    0, ...,   96,  138,   33],
       ..., 
       [   0,    0,    0, ..., 2052,   44, 2053],
       [   0,    0,    0, ...,    0,    9, 2054],
       [   0,    0,    0, ...,    5,  458,  665]])

In [140]:
y_train.values

array([0, 1, 0, ..., 0, 1, 1], dtype=int64)

In [141]:
new_X_test

array([[   0,    0,    0, ...,    0,    7,   58],
       [   0,    0,    0, ...,   24,   31,   23],
       [   0,    0,    0, ...,    0,    0, 1828],
       ..., 
       [   0,    0,    0, ...,    0,  312,   12],
       [   0,    0,    0, ...,  154,  104,  125],
       [   0,    0,    0, ...,  161,  744,    6]])

In [194]:
new_Y_train = to_categorical(y_train)
new_Y_train
#len(vocab)

array([[ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [197]:
new_Y_test = to_categorical(y_test)
new_Y_test

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,

In [207]:
def precision(y_true, y_pred):
    """Precision metric.
 
    Only computes a batch-wise average of precision.
 
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
 
def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

MODEL

In [214]:
model = Sequential()
model.add(Embedding(2055, 32, input_length=pad_length))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', precision, recall])

In [215]:
model.fit(new_X_train, new_Y_train, validation_split = 0.16, epochs = 10)

Train on 2522 samples, validate on 481 samples
Epoch 1/10
2522/2522 [==============================] - 4s - loss: 0.6473 - acc: 0.6649 - precision: 0.6674 - recall: 0.6646 - val_loss: 0.5065 - val_acc: 0.8119 - val_precision: 0.8199 - val_recall: 0.8004
Epoch 2/10
2522/2522 [==============================] - 1s - loss: 0.3574 - acc: 0.8731 - precision: 0.8736 - recall: 0.8723 - val_loss: 0.3302 - val_acc: 0.8617 - val_precision: 0.8610 - val_recall: 0.8628
Epoch 3/10
2522/2522 [==============================] - 1s - loss: 0.2273 - acc: 0.9270 - precision: 0.9281 - recall: 0.9259 - val_loss: 0.3035 - val_acc: 0.8888 - val_precision: 0.8867 - val_recall: 0.8919
Epoch 4/10
2522/2522 [==============================] - 2s - loss: 0.1588 - acc: 0.9477 - precision: 0.9473 - recall: 0.9481 - val_loss: 0.2977 - val_acc: 0.8857 - val_precision: 0.8887 - val_recall: 0.8815
Epoch 5/10
2522/2522 [==============================] - 2s - loss: 0.1172 - acc: 0.9625 - precision: 0.9627 - recall: 0.9623 

In [262]:
scores = model.evaluate(new_X_test, new_Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
y_pred = model.predict(new_X_test)
#y_pred
#precision([np.argmax(x) for x in new_Y_test], [np.argmax(x) for x in y_pred])
[p, r, f, _] = precision_recall_fscore_support([np.argmax(x) for x in new_Y_test], [np.argmax(x) for x in y_pred], average='binary')
#[[int(x[0].round()), int(x[1].round())] for x in y_pred]
#new_Y_test
#[np.argmax(x) for x in y_pred]
print("Precision: %.2f%%" % (p*100))
print("Recall: %.2f%%" % (r*100))
print("F-score: %.2f%%" % (f*100))

Accuracy: 87.45%
Precision: 83.05%
Recall: 89.91%
F-score: 86.34%


In [203]:
review = 'good one'
review = [review]
#print(review)
to_check = tokenizer.texts_to_sequences(review)
to_check = pad_sequences(to_check, maxlen=pad_length)
to_check

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  9, 39]])

In [206]:
predict_class = model.predict(to_check)
print(predict_class)
label = np.argmax(predict_class) 
print("Class : ", label)

if(label == 1):
    print("Positive")
else:
    print("Negative")


[[ 0.02560367  0.97549033]]
Class :  1
Positive


In [205]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 22, 32)            65760     
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 258       
Total params: 148,450
Trainable params: 148,450
Non-trainable params: 0
_________________________________________________________________
